In [1]:
# Binomial Pricing Model
# 1. Construction of Short Rate Lattice
# 2. Derivation of Elemantary Pricing Lattice from Short Rate Lattice
# 3. Pricing the Zero-coupon bond prices from Elemntary Pricing Lattice
# 4. Deriving spot rates from Zero-coupon bond prices
# 4. Construction of Swap Lattice from Short Rate Lattice
# 5. Pricing Swaptions

In [2]:
import numpy as np

In [3]:
n = 13 # number of periods
r_00 = 0.06
u = 1.25  
d = 0.9
q = 0.5
short_rate = np.zeros([n+1,n+1])

In [4]:
#Short-Rate Lattice
for i in range(n+1):
    for j in range(i+1):
        short_rate[j,i] = r_00 * u**(i-j) * d**(j)

In [5]:
#Elemantary Pricing Lattice
elem_price = np.zeros([n+2,n+2])
elem_price[0][0] = 1.0
for i in range(1,n+2):
    for j in range(i+1):
        if j == 0:
            elem_price[j,i] = elem_price[j,i-1]*q / (1+short_rate[j,i-1])
        elif j == i:
            elem_price[j,i] = elem_price[j-1,i-1]*(1-q) / (1+short_rate[j-1,i-1])
        else:
            elem_price[j,i] = (elem_price[j,i-1]*q / (1+short_rate[j,i-1])) + (elem_price[j-1,i-1]*(1-q)/ (1+short_rate[j-1,i-1]))

In [6]:
#Zero-Coupon bond prices can be derived from Elemantary Pricing Lattice
zcb = np.zeros([n+2])
for i in range(1,n+2):
    zcb[i] = sum(elem_price[:,i])

In [7]:
#Spot_rates can be calculated from the derived Zero-coupon bond prices
spot_rate = np.zeros([n+2])
for i in range(1,n+2):
    spot_rate[i] = ((1.0/zcb[i])**(1/i))-1

In [8]:
#Zero-coupon bond prices
FaceValue = float(input("Enter the face value of the Zero-coupon Bond: "))
print("Face Value of the bond =",round(FaceValue,2))
for i in range(1,n+2):
    print("Price of Zero-coupon bond of duration",i,"period(s) =",round(zcb[i]*FaceValue,2))

Enter the face value of the Zero-coupon Bond: 110
Face Value of the bond = 110.0
Price of Zero-coupon bond of duration 1 period(s) = 103.77
Price of Zero-coupon bond of duration 2 period(s) = 97.5
Price of Zero-coupon bond of duration 3 period(s) = 91.2
Price of Zero-coupon bond of duration 4 period(s) = 84.94
Price of Zero-coupon bond of duration 5 period(s) = 78.75
Price of Zero-coupon bond of duration 6 period(s) = 72.67
Price of Zero-coupon bond of duration 7 period(s) = 66.75
Price of Zero-coupon bond of duration 8 period(s) = 61.02
Price of Zero-coupon bond of duration 9 period(s) = 55.54
Price of Zero-coupon bond of duration 10 period(s) = 50.32
Price of Zero-coupon bond of duration 11 period(s) = 45.39
Price of Zero-coupon bond of duration 12 period(s) = 40.78
Price of Zero-coupon bond of duration 13 period(s) = 36.49
Price of Zero-coupon bond of duration 14 period(s) = 32.53


In [9]:
#Spot Rates
for i in range(1,n+2):
    print("Spot Rate in period",i,"=",round(spot_rate[i]*100,2),"%")

Spot Rate in period 1 = 6.0 %
Spot Rate in period 2 = 6.22 %
Spot Rate in period 3 = 6.45 %
Spot Rate in period 4 = 6.68 %
Spot Rate in period 5 = 6.91 %
Spot Rate in period 6 = 7.15 %
Spot Rate in period 7 = 7.4 %
Spot Rate in period 8 = 7.64 %
Spot Rate in period 9 = 7.89 %
Spot Rate in period 10 = 8.14 %
Spot Rate in period 11 = 8.38 %
Spot Rate in period 12 = 8.62 %
Spot Rate in period 13 = 8.86 %
Spot Rate in period 14 = 9.09 %


In [10]:
#Swap Lattice
swap_rate = float(input("Enter Swap Fixed Rate (in %): "))
swap_rate = swap_rate/100
swap_t = int(input("Enter Swap Expiration period: "))
swap = np.zeros([swap_t,swap_t])
swap[:,swap_t-1] = (short_rate[0:swap_t,swap_t-1] - swap_rate)/(1+short_rate[0:swap_t,swap_t-1])

for i in range(swap_t-2,-1,-1):
    for j in range(i+1):
        swap[j,i] = ((short_rate[j,i] - swap_rate) + swap[j,i+1]*q + (1-q)*swap[j+1,i+1]) / (1+short_rate[j,i])

Enter Swap Fixed Rate (in %): 5
Enter Swap Expiration period: 7


In [11]:
#Swaption Pricing
swaption_strike = 0.00     #generally 0
swaption_t = int(input("Enter Swaption Expiration period: "))
p = float(input("Enter principal amount: "))
swaption = np.zeros([swaption_t+1,swaption_t+1])
swaption[:,swaption_t] = np.maximum(np.zeros([swaption_t+1]),swap[0:swaption_t+1,swaption_t])
for i in range(swaption_t-1,-1,-1):
    for j in range(i+1):
        swaption[j,i] = (swaption[j,i+1]*q + (1-q)*swaption[j+1,i+1]) / (1+short_rate[j,i])
print("Swaption Price:",round(p*swaption[0,0],2))

Enter Swaption Expiration period: 4
Enter principal amount: 1000000
Swaption Price: 67316.1
